## 金十日历查询

In [1]:
import datetime
import json
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
url = rf'https://cdn-rili.jin10.com/web_data/2023/daily/09/21/economics.json'
response = requests.get(url)
data = json.loads(response.text)

# 国际时间转化为北京时间
def trs_time(x):
    datetime_object = datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')
    datetime_object = datetime_object + timedelta(hours=8)  # 转化为北京时间
    return datetime_object
df_json = pd.DataFrame(data)
df_json['pub_time'] = df_json['pub_time'].apply(trs_time)
df_json = df_json[['pub_time','actual','consensus','previous','unit','country','name','indicator_id']]
df_json[df_json['country'] == '美国']

## 金十All_Index字段更新

In [38]:
import datetime
import json
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# 查询数据库对应表，转化为dataframe
def read_db_df(table_name='All_Index'):
    engine_params = {
        'dialect': 'mssql',
        'driver': 'pymssql',
        'username': 'sa',
        'password': 'Goldriven123',
        'host': 'sh1.goldriven.com',
        'port': 1433,
        'database': 'JINSHI_FCT'
    }
    
    # Create the SQLAlchemy engine
    engine = create_engine(
        "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}?charset=utf8"
        .format(**engine_params))
    
    # SQL query to get the data from the table
    query = f"SELECT * FROM {table_name};"  #sql语言查询所有数据
    
    # Use pandas to read the data from the database into a DataFrame
    df = pd.read_sql(query, engine)
    return df

#将本地添加到数据库
def append_df_to_db_2(df,table_name):
    engine_params = {
        'dialect': 'mssql',
        'driver': 'pymssql',
        'username': 'sa',
        'password': 'Goldriven123',
        'host': 'sh1.goldriven.com',
        'port': 1433,
        'database': 'JINSHI_FCT'
    }
    engine = create_engine(
        "{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}?charset=utf8"
        .format(**engine_params))
    try:
        name_1 = '['+table_name+']' if '-' in table_name else table_name
        #session = sessionmaker(bind=engine)
        with engine.connect() as connection:
            delete_query = f"DROP TABLE {name_1}"
            #delete_query = f"DELETE FROM {name_1}"
            connection.execute(delete_query)
            
        df.to_sql(con=engine,name=table_name,if_exists='append',index=False)
        print(f"{table_name} is successfully stored in the database: {engine_params['database']}")
    except:
        df.to_sql(con=engine,name=table_name,if_exists='append',index=False)
        print(f"{table_name} is successfully stored in the database: {engine_params['database']}")